# Wikiからの情報抽出
TOP-Kチャンクの要約

## 定数情報

In [ ]:
# ディレクトリ
DATA_DIR = "./llamaindex/data/wiki"
PERSIST_DIR = "./llamaindex/storage/wiki"

# プロンプト
Information_Extraction_Prompt = "「処理速度に関する性能」についてのノウハウを教えて下さい。"
#Information_Extraction_Prompt = "「業務システムの移行マイグレーション」についてのノウハウ"

Information_Summary_Prompt = "以下は「処理速度に関する性能」についてのノウハウを含むチャンクです。この中から、レポート化を見据えて「処理速度に関する性能」のノウハウを抽出してください。"
#Information_Summary_Prompt = "以下は「業務システムの移行マイグレーション」についてのノウハウを含むチャンクです。この中から、レポート化を見据えて「処理速度に関する性能」のノウハウを抽出してください。"

# チャンク数
Similarity_TopK = 1000
# 類似度しきい値
Similarity_Threshold = 0.8
# ウィンドウサイズ（文字数）
Window_Size = 8000

## ライブラリ

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings

In [ ]:
# 評価の所で使う
from llama_index.llms.openai import OpenAI

## キー

In [ ]:
import os
print(os.environ['OPENAI_API_KEY'])

## ベクトル検索

### 準備

In [ ]:
import os.path
from llama_index.core import (
    StorageContext,
    load_index_from_storage,
)

# check if storage already exists
if not os.path.exists(PERSIST_DIR):
    print("load the documents and create the index")
    documents = SimpleDirectoryReader(DATA_DIR, recursive=True).load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    print("load the existing index")
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

### 試行
通常のRAG（ベクトル検索のTOP-K）は、情報抽出の用途には適合しない。

In [ ]:
# Either way we can now query the index
query_engine = index.as_query_engine()
response = query_engine.query(Information_Extraction_Prompt)
print(response)

### Retrieve
ベクトル検索のTOP-KをチューニングしてRetrieveする。

In [ ]:
retriever = index.as_retriever(similarity_top_k=Similarity_TopK) # 多目に指定
nodes = retriever.retrieve(Information_Extraction_Prompt)
# print(response.source_nodes) # プロパティを確認したい場合、ココを実行して出力を確認

### Nodesチェック

#### PukiWikiのダンプファイル名のデコード
一部上手くいかないことがある。

In [ ]:
###################################################
# pwdecode.py :
#      Ver.0 2024-02-23
# ■ 概要
#     PukiWiki の wiki/ 以下のファイル名はエンコードされている。
#     これを読めるようにする。実用上は、ls を使うはずなので、
#     ls に対してパイプラインで実行することを想定している。
# ■ 使用例
#     ls -ltr | cat -n | pwdecode.py
#     ※ cat -n を入れておくと、実際のファイル名と簡単に対応付けられる。
###################################################
import sys
import re

# PukiWikiは、utf-8 と euc_jp とがある。成功した方を採用。
def try_decode(encoded_bytes):
    try:
        return encoded_bytes.decode('euc_jp')
    except UnicodeDecodeError:
        pass  

    try:
        return encoded_bytes.decode('utf-8')
    except UnicodeDecodeError:
        return None

# 各行のデコード
def decode_pukiwiki_filename(encoded_strings):
    decoded_filename = ''    
    hex_pattern = re.compile(r'([0-9A-F]{4,})(?=\.txt$)') # エンコードされたファイル名部分を抽出
    pos = 0

    for match in hex_pattern.finditer(encoded_strings):
        start, end = match.span()
        decoded_filename += encoded_strings[pos:start]  # 16進数でない部分を追加
        
        hex_str = match.group(1)
        decoded_bytes = bytes.fromhex(hex_str)
        decoded_part = try_decode(decoded_bytes)
        if decoded_part is not None:
            decoded_filename += decoded_part
        else:
            decoded_filename += hex_str  # デコードに失敗した場合は元の16進数の文字列を追加
        pos = end

    decoded_filename += encoded_strings[pos:]  # 残りの部分を追加

    return decoded_filename

### Nodes情報のダンプ
内容を確認する際は、コメントアウトを解除する。

In [ ]:
import codecs

for i, node in enumerate(nodes, start=1):  # start=1で1から始まる

    if node.score < Similarity_Threshold:
        break
    """  
    print("No.", i)    
    print("score ", node.score)
    print("id_", node.id_)
    print("file_name", decode_pukiwiki_filename(node.metadata["file_name"]))
    print("text", node.text) # テキストを確認したい場合、ココを実行して出力を確認
    print("------------------------------------------------------")
    """

## 情報の要約

### TOP-Kチャンクの要約

In [ ]:
def CallLLM(Prompt):
    llm = OpenAI(model="gpt-4o")
    response = llm.complete(Prompt)
    return response

In [ ]:
buffer = []
buffer_len = 0

for i, node in enumerate(nodes, start=1):
    
    if node.score < Similarity_Threshold:
        break
        
    text = node.text
    text_len = len(text)

    if buffer_len + text_len <= Window_Size:
        # バッファを蓄積
        i=i
    else:
        # バッファを処理
        print(CallLLM(Information_Summary_Prompt  + "\n\n\n" + str(buffer)))
        print("------------------------------------------------------")
        # バッファを初期化
        buffer = []
        buffer_len = 0
        
    # バッファに追記
    buffer.append(node)
    buffer_len += text_len
    
# 最後に残ったバッファも忘れずに処理
print(CallLLM(Information_Summary_Prompt  + "\n\n\n" + str(buffer)))

### 要約の構造化
- 構造化はドメイン知識を要する。
- リーフページにスコープ外の情報が多いと要約がなかなか上手く行かない。

In [ ]:
# ...